In [86]:
import pandas as pd
import numpy as np

In [87]:
ret = pd.read_csv('../data/mid_cap_all_sectors_ret.csv', index_col='date') * 100
vol = pd.read_csv('../data/mid_cap_all_sectors_volume.csv', index_col='date')

In [88]:
combined_array = np.stack((ret.values, vol.values), axis=-1)

combined_data = [[list(combined_array[i, j]) for j in range(combined_array.shape[1])] for i in range(combined_array.shape[0])]

df_combined = pd.DataFrame(combined_data, index=ret.index, columns=ret.columns)

In [89]:
df_combined

,Materials,Industrials,Health Care,Real Estate,Consumer Discretionary,Financials,Utilities,Information Technology,Energy,Consumer Staples,Communication Services
date,,,,,,,,,,,
2000-01-04,"[-1.11525043846327, 345388.95238095237]","[-2.40561818988466, 396588.4042553191]","[-1.26468336875092, 333422.8125]","[-0.79964385581844, 98782.5625]","[-2.58734692042006, 694893.75]","[-2.93774041598001, 235307.6744186046]","[-0.754774684575, 123696.45454545454]","[-3.95670656356584, 738020.9090909092]","[-0.65655703931916, 333797.625]","[-1.49546769359482, 203379.125]","[-4.0996009242407805, 384625.0]"
2000-01-05,"[1.4502987477337401, 457444.0952380952]","[0.9858210351485499, 441508.7142857142]","[-0.13901034940271, 256259.11764705885]","[0.8318398529225199, 109816.0]","[0.57109068762815, 837709.375]","[-0.39320625560758, 207513.32558139536]","[2.0653855988139203, 490148.5454545454]","[1.3641597783172499, 1160541.181818182]","[0.12763897814911002, 406316.0]","[1.51556186276755, 181086.0]","[-1.0760836613009999, 420875.0]"
2000-01-06,"[1.91512011421743, 480816.5238095238]","[-0.41400462105219, 339793.4285714285]","[1.7520144716326398, 376177.7647058823]","[1.3901631253156599, 107236.9375]","[-0.0759509258055, 1242315.625]","[0.7219254628857099, 185944.6511627907]","[0.57343640598296, 139097.45454545456]","[-1.7140802931160701, 1168490.7272727273]","[1.70087052084814, 342886.875]","[-1.22179222509644, 306957.625]","[-0.8095716745474, 315925.0]"
2000-01-07,"[0.14631412967731, 408331.38095238095]","[0.8534238976123101, 389937.5625]","[4.70341195715487, 403742.17647058825]","[1.1037948699275701, 171373.0625]","[1.0003990606722, 926128.125]","[1.1716713721085599, 219475.8809523809]","[0.97062529783078, 192216.6363636364]","[2.59195806291758, 1235088.1363636365]","[1.90339746693176, 586391.5]","[1.02838458760443, 474322.5714285715]","[0.41642849712824, 341050.0]"
2000-01-10,"[0.23547467301694003, 385219.5238095238]","[0.9016368644763999, 431528.612244898]","[3.2520534432928696, 393175.29411764705]","[0.72168097290211, 112760.8125]","[0.12982212064606, 806553.125]","[-0.56581776545639, 161326.7441860465]","[-0.7170271277632501, 125774.1818181818]","[5.1412222307377204, 821543.4545454546]","[-0.2799953961342, 215876.125]","[-0.19414748563956, 203777.875]","[2.46858859705642, 487500.0]"
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,"[0.25356148751893004, 1545215.5357142854]","[-0.12401287311640001, 469179.2465753424]","[-0.45471232620393, 978310.8157894736]","[-0.062022240657719996, 1106939.3928571427]","[-0.5528864529642, 1275309.78125]","[-0.54557931753431, 911245.8059701492]","[0.38494845361012003, 586447.6666666667]","[-0.96827987403466, 879552.9268292682]","[0.24708830818755, 2121710.0]","[-0.37936529210089, 1275695.6470588234]","[0.62408267232889, 1326355.888888889]"
2024-08-27,"[-0.7176392821917099, 1629227.7142857143]","[-0.73018529831534, 479813.0]","[0.042967263495149996, 817449.9210526313]","[0.10187021985654, 1052618.4642857143]","[-0.30795359668325, 1262377.859375]","[-0.30756431201051004, 759084.2537313433]","[-0.69419292014114, 439154.6]","[-0.22368158937494, 802428.536585366]","[-1.4483494353172, 2071944.2]","[-0.41336402706524994, 1145564.9411764706]","[-0.35795541751246, 1075439.2222222222]"
2024-08-28,"[-1.01682643139978, 1511510.964285714]","[-0.5247582210971, 511664.1780821918]","[-0.6335748766578899, 1110610.3684210526]","[-0.04512958789092, 1019164.5714285714]","[-1.33746829218429, 1598482.140625]","[0.6221879161696, 838194.8507462685]","[0.01621408664049, 527750.2]","[-1.37159640144655, 1081261.951219512]","[-0.85689381207924, 2048294.9]","[-0.22251496834107, 1177785.9411764704]","[-0.9943968725076, 1295391.8888888888]"


In [90]:
import torch
def data_to_tensor(data:pd.DataFrame, dtype=torch.float32):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    returns = data.map(lambda x: x[0]).to_numpy() 
    volumes = data.map(lambda x: x[1]).to_numpy()

    full = np.stack([returns, volumes], axis=1)
    full = full.transpose(1, 2, 0)

    return torch.tensor(full, dtype=dtype).to(device)

class CNNDataset(torch.utils.data.Dataset):
    def __init__(self, data: pd.Series, seq_n: int) -> None:
        sample_index = data.shift(seq_n-1).dropna().index.tolist()
        self.data_list = []
        for sample in sample_index:
            data_tensor = data_to_tensor(data.loc[:sample].iloc[-seq_n:])
            data_tuple = (data_tensor, data_tensor)
            self.data_list.append(data_tuple)

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, index):
        return self.data_list[index]

In [91]:
data = CNNDataset(df_combined, seq_n=100)

In [112]:
len(data)

6105

In [106]:
data[1][0].shape

torch.Size([2, 11, 100])

In [111]:
data[1][0][1,6,:]

tensor([490148.5312, 139097.4531, 192216.6406, 125774.1797,  96273.3672,
        107800.1797, 194852.8125, 159132.0938, 148186.9062, 105977.8203,
        216158.0000, 377761.8125, 411681.2812, 321685.0938, 270123.3750,
        147862.1875, 275104.9062, 206880.5469, 144185.3594, 147168.1875,
        142272.3594, 149864.1875, 125842.5469, 176433.3594, 146178.1875,
        191610.0000, 176643.0938, 171646.6406, 166339.0000, 125561.0000,
        184904.0000, 130066.1797, 160961.9062, 187045.7344, 140920.7344,
        102114.6328, 183285.7344, 187347.4531, 126277.8203, 177575.2656,
        123587.5469, 126114.4531, 179202.2656, 207844.7344, 145117.4531,
        145461.2656, 184299.2656, 221692.6406, 174597.5469, 161647.0000,
        147645.0938, 100803.8203, 107553.9062, 104558.4531, 145994.6406,
        151223.8125, 139438.0938, 122388.8203, 245482.7344, 121870.1797,
        410397.5312, 171183.7344, 176892.0938, 536751.7500, 266606.7188,
        168539.7344, 129135.0938, 113351.0000, 1544

In [108]:
ret.iloc[1:101,6]

date
2000-01-05    2.065386
2000-01-06    0.573436
2000-01-07    0.970625
2000-01-10   -0.717027
2000-01-11   -0.675110
                ...   
2000-05-22    1.114101
2000-05-23   -0.870239
2000-05-24    0.867466
2000-05-25   -0.859089
2000-05-26    1.661480
Name: Utilities, Length: 100, dtype: float64

In [110]:
vol.iloc[1:101,6]

date
2000-01-05    490148.545455
2000-01-06    139097.454545
2000-01-07    192216.636364
2000-01-10    125774.181818
2000-01-11     96273.363636
                  ...      
2000-05-22    106098.727273
2000-05-23    104161.818182
2000-05-24    175216.363636
2000-05-25    101497.363636
2000-05-26    123312.636364
Name: Utilities, Length: 100, dtype: float64